In [1]:
import pandas as pd
import numpy as np
!pip install folium
import folium
!pip install geocoder
import geocoder
import json
import urllib.request
!pip install geopandas
import geopandas as gpd

     |████████████████████████████████| 92kB 14.7MB/s eta 0:00:01
     |████████████████████████████████| 102kB 2.4MB/s ta 0:00:01
     |████████████████████████████████| 901kB 15.2MB/s eta 0:00:01
     |████████████████████████████████| 10.1MB 43.5MB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 42.1MB/s eta 0:00:01
     |████████████████████████████████| 17.9MB 35.2MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built munch


In [80]:
mestky_casty_url = 'http://opendata.iprpraha.cz/CUR/DTMP/TMMESTSKECASTI_P/S_JTSK/TMMESTSKECASTI_P.json'

with urllib.request.urlopen(mestky_casty_url) as raw_data:
    data = raw_data.read()
    mestky_casty_json = json.loads(data, encoding = "ISO 8859-2")

df_mestky_casty_geo = pd.DataFrame(mestky_casty_json["features"])

In [3]:
g = geocoder.arcgis('Prague')
lat = g.json['lat']
lng = g.json['lng']
print('Prague coordinates {} {}'.format(lat, lng))

Prague coordinates 50.07913000000008 14.433020000000056


In [81]:
df_mestky_casty_geo.head()

,geometry,properties,type
0,"{'type': 'Polygon', 'coordinates': [[[-750495....","{'OBJECTID': 1, 'DAT_VZNIK': '20180105160833',...",Feature
1,"{'type': 'Polygon', 'coordinates': [[[-743677....","{'OBJECTID': 2, 'DAT_VZNIK': '20180910110223',...",Feature
2,"{'type': 'Polygon', 'coordinates': [[[-752038....","{'OBJECTID': 3, 'DAT_VZNIK': '20181204155900',...",Feature
3,"{'type': 'Polygon', 'coordinates': [[[-738597....","{'OBJECTID': 4, 'DAT_VZNIK': '20170817145228',...",Feature
4,"{'type': 'Polygon', 'coordinates': [[[-744523....","{'OBJECTID': 5, 'DAT_VZNIK': '20181009145125',...",Feature


In [4]:
df_mestky_casty_geo.shape

(57, 3)

'PLOCHA': 9902539.83,
    'ID': 13,
    'KOD_MC': 539635,
    'NAZEV_MC': 'Praha-Řeporyje',
    'KOD_MO': 51,
    'KOD_SO': '132',
    'TID_TMMESTSKECASTI_P': 13,
    'POSKYT': 'HMP-IPR',
    'ID_POSKYT': 43,
    'STAV_ZMENA': 'U',
    'NAZEV_1': 'Řeporyje',
    'Shape_Length': 24169.831485276492,
    'Shape_Area': 9902539.83125}},

In [5]:
praha_parks_url = 'http://opendata.praha.eu/dataset/1b9c47a0-518f-435b-a4be-bc5814674b02/resource/153996b7-f456-497d-816c-0ad84bcb5547/download/03f2c696-ad63-4dc5-94c6-d22c0ebff3ff-gardens.json'
with urllib.request.urlopen(praha_parks_url) as raw_parks:
    data_praks = raw_parks.read()
    parks_json = json.loads(data_praks)   
parks_json     

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'id': 'betramka',
    'name': 'Betramka',
    'image': 'http://www.praha.eu/public/5c/7c/31/96820_4_bertramka_08.jpg',
    'description': 'Mozartova busta v zahradě legendární usedlosti stojí na vyvýšeném, trochu zvláštním, místě, jakoby stranou pomyslné hlavní osy podlouhlého pozemku. Mistr, byť pouze v kamenné podobě, tak má dokonalý přehled o dění v zahradě, ale současně překrásný výhled na stráně vrchu Mrázovka.',
    'url': 'http://www.praha.eu/jnp/cz/co_delat_v_praze/parky/bertramka/bertramka_text.html',
    'address': 'Mozartova 844/14',
    'district': 'praha-5',
    'properties': [{'id': 'wc', 'value': 'ANO'},
     {'id': 'misto',
      'value': 'Kulturní památka, koncerty vážné hudby, společenské akce, muzeum W. A. Mozarta'},
     {'id': 'kolo', 'value': 'ANO'},
     {'id': 'hriste', 'value': 'NE'},
     {'id': 'brusle', 'value': 'NE'},
     {'id': 'sport', 'value': 'NE'},
     {'id': 'mhd', 'va

In [6]:
hriste = 'http://opendata.praha.eu/dataset/3c3ca9ca-fbc0-4f97-b624-ed967f5d9a24/resource/e19c2e29-5e33-4449-8847-5dc8f5b8a2f2/download/db144c03-1a0f-456f-a32b-9c48ccfc0813-playgrounds.json'
with urllib.request.urlopen(hriste) as raw_hriste:
    data_hriste = raw_hriste.read()
    hriste_json = json.loads(data_hriste)

In [7]:
prague_map = folium.Map(location=[lat, lng], zoom_start = 14)

for hriste in hriste_json['features']:
    geom = hriste['geometry']
    props = hriste['properties']
    folium.Marker(
    [float(geom['coordinates'][1]),float(geom['coordinates'][0])],
    icon = folium.Icon(color='pink',icon='star'),
    popup=props['name']).add_to(prague_map)
    
for park in parks_json ['features']:
    geom =park['geometry']
    props = park['properties']
    folium.Marker(
    [float(geom['coordinates'][1]),float(geom['coordinates'][0])],
    icon = folium.Icon(color='green',icon='tree-deciduous'),
    popup=props['name']).add_to(prague_map)
    
prague_map


In [68]:
url_pocet =  'https://www.czso.cz/documents/10180/25233177/sldb_zv.csv'

df_lidi = pd.read_csv(url_pocet,encoding = "ISO 8859-2")
df_lidi.head()

,typuz_naz,nazev,uzcis,uzkod,u01,u02,u03,u04,u05,u06,u07,u08,u09,u10,u11
0,kraj,Hlavní město Praha,100,3018,1268796.0,613738.0,655058.0,153622.0,908321.0,201029.0,644643.0,600730.0,92927.0,542168.0,579509.0
1,kraj,Středočeský kraj,100,3026,1289211.0,637252.0,651959.0,199300.0,895024.0,190911.0,639851.0,587539.0,286780.0,482860.0,523045.0
2,kraj,Jihočeský kraj,100,3034,628336.0,308296.0,320040.0,91119.0,435187.0,100000.0,307130.0,280844.0,123048.0,247608.0,262692.0
3,kraj,Plzeňský kraj,100,3042,570401.0,282137.0,288264.0,79469.0,396468.0,92734.0,278674.0,255278.0,105835.0,226298.0,242397.0
4,kraj,Karlovarský kraj,100,3051,295595.0,145483.0,150112.0,42159.0,207480.0,44538.0,139871.0,123100.0,39845.0,119403.0,128904.0


In [92]:
df_population_praha = df_lidi[(df_lidi.uzcis == 44)& (df_lidi.nazev.str.find('Praha') != -1)][['nazev','u01','u04', 'u05', 'u06']]
df_population_praha.rename(columns={'nazev':'Name','u01':'Total', 'u04':'Kids', 'u05':'Middle', 'u06':'Senior'}, inplace = True)
df_population_praha['Name'] = df_population_praha['Name'].str.lower()
df_population_praha.head()

,Name,Total,Kids,Middle,Senior
6342,praha 1,30561.0,2391.0,22963.0,4594.0
6343,praha 2,48773.0,4909.0,36293.0,7322.0
6344,praha 3,71769.0,7111.0,52275.0,12010.0
6345,praha 4,131793.0,13793.0,88642.0,28798.0
6346,praha 5,83968.0,10571.0,61294.0,11668.0


In [77]:
df_population_praha.shape

(57, 5)

In [91]:
df_population_praha['Name'].unique()

array(['Praha 1', 'Praha 2', 'Praha 3', 'Praha 4', 'Praha 5', 'Praha 6',
       'Praha 7', 'Praha 8', 'Praha 9', 'Praha 10', 'Praha-Běchovice',
       'Praha-Benice', 'Praha-Březiněves', 'Praha-Dolní Počernice',
       'Praha-Dubeč', 'Praha 20', 'Praha-Klánovice', 'Praha-Koloděje',
       'Praha-Kolovraty', 'Praha-Královice', 'Praha-Křeslice',
       'Praha-Nedvězí', 'Praha-Satalice', 'Praha 22', 'Praha 21',
       'Praha-Vinoř', 'Praha-Lipence', 'Praha-Lochkov',
       'Praha-Přední Kopanina', 'Praha 16', 'Praha-Řeporyje',
       'Praha-Slivenec', 'Praha 13', 'Praha-\x8aeberov', 'Praha-Újezd',
       'Praha-Zbraslav', 'Praha-Zličín', 'Praha 11', 'Praha-Kunratice',
       'Praha-Libu\x9a', 'Praha 12', 'Praha-Velká Chuchle',
       'Praha-Lysolaje', 'Praha-Nebu\x9aice', 'Praha 17', 'Praha-Suchdol',
       'Praha-Ďáblice', 'Praha-Dolní Chabry', 'Praha-Čakovice',
       'Praha-Troja', 'Praha 19', 'Praha 14', 'Praha-Dolní Měcholupy',
       'Praha 15', 'Praha-Petrovice', 'Praha-\x8atěrboho

In [24]:
df_population_praha.describe()

,Total,Kids,Middle,Senior
count,57.000000,57.000000,57.000000,57.000000
mean,22259.578947,2695.122807,15935.456140,3526.824561
std,33174.735475,3707.926471,23224.196509,6317.295585
min,318.000000,44.000000,213.000000,38.000000
25%,2527.000000,432.000000,1869.000000,322.000000
50%,5218.000000,784.000000,3875.000000,561.000000
75%,30561.000000,3070.000000,22963.000000,4002.000000
max,131793.000000,13793.000000,88642.000000,28798.000000


In [82]:
neighborhood = pd.DataFrame(columns=['Id', 'neighborhood', 'geometry', 'area'])

In [83]:
index = 0
for index, row in df_mestky_casty_geo.iterrows():
    neighborhood.loc[index] = [row['properties']['ID'], row['properties']['NAZEV_MC'].lower(), row['geometry'], row['properties']['PLOCHA']]


neighborhood.head()


,Id,neighborhood,geometry,area
0,13,praha-řeporyje,"{'type': 'Polygon', 'coordinates': [[[-750495....",9902539.83
1,43,praha 1,"{'type': 'Polygon', 'coordinates': [[[-743677....",5538443.86
2,1,praha 6,"{'type': 'Polygon', 'coordinates': [[[-752038....",41561204.34
3,10,praha 9,"{'type': 'Polygon', 'coordinates': [[[-738597....",13310394.12
4,44,praha-suchdol,"{'type': 'Polygon', 'coordinates': [[[-744523....",5135601.75


In [53]:
len(neighborhood['neighborhood'].unique())

57

In [129]:
coulumns = ['Id','neighborhood','Total', 'Kids', 'Middle', 'Senior','geometry', 'area']
df_prague = neighborhood.merge(df_population_praha, left_on='neighborhood', right_on ='Name', how='left')[coulumns]
df_prague.shape

(57, 8)

In [130]:
df_prague.dropna(inplace =True)
df_prague.shape

(53, 8)

In [111]:
dict_coordinates = {}
for index, row in df_prague.iterrows():
    g = geocoder.arcgis(row['neighborhood'])
    lat = g.json['lat']
    lng = g.json['lng']
    dict_coordinates[row['neighborhood']] = [lat, lng]
    print('{} - lat {}, long {}'.format(row['neighborhood'],lat, lng)) 

praha-řeporyje - lat 50.03322000000003, long 14.311950000000024
praha 1 - lat 50.08728000000008, long 14.41742000000005
praha 6 - lat 50.094160000000045, long 14.367480000000057
praha 9 - lat 50.11041000000006, long 14.58059000000003
praha-suchdol - lat 50.13701000000003, long 14.369320000000073
praha-dolní chabry - lat 50.146250000000066, long 14.448260000000062
praha-petrovice - lat 50.036110000000065, long 14.562420000000031
praha-čakovice - lat 50.15146000000004, long 14.526340000000062
praha 14 - lat 50.10682000000003, long 14.55731000000003
praha 12 - lat 50.00564000000003, long 14.404620000000023
praha 17 - lat 50.06535000000008, long 14.308060000000069
praha-ďáblice - lat 50.148430000000076, long 14.486430000000041
praha 10 - lat 50.060550000000035, long 14.557030000000054
praha-újezd - lat 50.01398000000006, long 14.54558000000003
praha-březiněves - lat 50.16591000000005, long 14.48389000000003
praha 2 - lat 50.07394000000005, long 14.439560000000029
praha 13 - lat 50.05163000

In [134]:
df_prague['latitude'] = 0.0
df_prague['longitude'] = 0.0
for k, v in dict_coordinates.items():
    df_prague.loc[df_prague.neighborhood == k,'latitude']=v[0]
    df_prague.loc[df_prague.neighborhood == k,'longitude']=v[1]

In [136]:
for index, row in df_prague.iterrows():
    folium.Marker(
    [row['latitude'],row['longitude'] ],
    icon = folium.Icon(color='red'),
    popup=row['neighborhood']).add_to(prague_map)
    
prague_map